In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6169,2020-12-13T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1226,198,1424,34295,...,1967,NaN,NaN,78190,1082424,727787.0,NaN,12.0,NaN,NaN
6170,2020-12-13T17:00:00,ITA,9,Toscana,43.769231,11.255889,1180,219,1399,16715,...,3151,NaN,NaN,112344,1718159,1005743.0,NaN,10.0,Positivi diagnosticati solo con test antigenic...,NaN
6171,2020-12-13T17:00:00,ITA,10,Umbria,43.106758,12.388247,294,45,339,4469,...,526,NaN,NaN,26190,456106,236951.0,NaN,1.0,NaN,NaN
6172,2020-12-13T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,74,8,82,540,...,353,NaN,NaN,6871,58056,35365.0,NaN,0.0,NaN,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6169,2020-12-13T17:00:00,19,Sicilia,1226,198,1424,34295,35719,-42,808,40504,1967,NaN,NaN,78190,1082424,727787.0,12.0,NaN,NaN
6170,2020-12-13T17:00:00,9,Toscana,1180,219,1399,16715,18114,-978,673,91079,3151,NaN,NaN,112344,1718159,1005743.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN
6171,2020-12-13T17:00:00,10,Umbria,294,45,339,4469,4808,-76,145,20856,526,NaN,NaN,26190,456106,236951.0,1.0,NaN,NaN
6172,2020-12-13T17:00:00,2,Valle d'Aosta,74,8,82,540,622,-41,34,5896,353,NaN,NaN,6871,58056,35365.0,0.0,NaN,NaN


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-12-13', '2020-12-12')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-13T17:00:00,1,Piemonte,3835,291,4126,51848,55974,-1887,1011,122476,7088,NaN,NaN,185538,1757469,1036540.0,6.0,Il numero dei tamponi antigenici positivi è so...,NaN
1,2020-12-13T17:00:00,2,Valle d'Aosta,74,8,82,540,622,-41,34,5896,353,NaN,NaN,6871,58056,35365.0,0.0,NaN,NaN
2,2020-12-13T17:00:00,3,Lombardia,5159,714,5873,80476,86349,1283,2335,333503,23810,NaN,NaN,443662,4438316,2464408.0,24.0,NaN,NaN
3,2020-12-13T17:00:00,5,Veneto,2606,340,2946,87753,90699,2567,4092,95141,4801,NaN,NaN,190641,2997581,1125556.0,13.0,NaN,NaN
4,2020-12-13T17:00:00,6,Friuli Venezia Giulia,655,60,715,13458,14173,-182,573,24683,1233,NaN,NaN,40089,815900,310500.0,3.0,NaN,NaN
5,2020-12-13T17:00:00,7,Liguria,768,75,843,7552,8395,-74,331,44566,2657,NaN,NaN,55618,649185,304928.0,4.0,i 113911 test antigenici rapidi sono riferiti ...,NaN
6,2020-12-13T17:00:00,8,Emilia-Romagna,2819,220,3039,61723,64762,790,1940,73421,6572,NaN,NaN,144755,2323522,1150185.0,14.0,NaN,NaN
7,2020-12-13T17:00:00,9,Toscana,1180,219,1399,16715,18114,-978,673,91079,3151,NaN,NaN,112344,1718159,1005743.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-13T17:00:00,10,Umbria,294,45,339,4469,4808,-76,145,20856,526,NaN,NaN,26190,456106,236951.0,1.0,NaN,NaN
9,2020-12-13T17:00:00,11,Marche,476,81,557,11737,12294,-309,481,21100,1404,NaN,NaN,34798,456349,267405.0,4.0,NaN,NaN


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-12T17:00:00,1,Piemonte,3857,304,4161,53700,57861,-2130,1443,119614,7052,NaN,NaN,184527,1749139,1032439.0,6.0,NaN,NaN
1,2020-12-12T17:00:00,2,Valle d'Aosta,76,8,84,579,663,-53,25,5823,351,NaN,NaN,6837,57784,35186.0,0.0,NaN,NaN
2,2020-12-12T17:00:00,3,Lombardia,5289,717,6006,79060,85066,-2383,2736,332595,23666,NaN,NaN,441327,4412793,2455811.0,26.0,NaN,NaN
3,2020-12-12T17:00:00,5,Veneto,2609,345,2954,85178,88132,3039,5098,93648,4769,NaN,NaN,186549,2981631,1120408.0,54.0,NaN,NaN
4,2020-12-12T17:00:00,6,Friuli Venezia Giulia,654,59,713,13642,14355,120,884,23946,1215,NaN,NaN,39516,809115,306882.0,8.0,NaN,NaN
5,2020-12-12T17:00:00,7,Liguria,782,77,859,7610,8469,-296,263,44169,2649,NaN,NaN,55287,646433,304092.0,4.0,i 112175 test antigenici rapidi sono riferiti ...,NaN
6,2020-12-12T17:00:00,8,Emilia-Romagna,2813,222,3035,60937,63972,-83,1807,72315,6529,NaN,NaN,142816,2312385,1144793.0,14.0,NaN,NaN
7,2020-12-12T17:00:00,9,Toscana,1221,228,1449,17643,19092,-1732,574,89462,3117,NaN,NaN,111671,1705743,1001493.0,11.0,Positivi diagnosticati solo con test antigenic...,NaN
8,2020-12-12T17:00:00,10,Umbria,301,49,350,4534,4884,-205,211,20644,517,NaN,NaN,26045,453903,236236.0,0.0,NaN,NaN
9,2020-12-12T17:00:00,11,Marche,487,86,573,12030,12603,-510,451,20324,1390,NaN,NaN,34317,452143,264878.0,2.0,NaN,NaN


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-12-13T17:00:00,1,Piemonte,3835,291,4126,51848,55974,-1887,1011,122476,7088,NaN,NaN,185538,1757469,1036540.0,6.0,Il numero dei tamponi antigenici positivi è so...,NaN
1,2020-12-13T17:00:00,2,Valle d'Aosta,74,8,82,540,622,-41,34,5896,353,NaN,NaN,6871,58056,35365.0,0.0,NaN,NaN
2,2020-12-13T17:00:00,3,Lombardia,5159,714,5873,80476,86349,1283,2335,333503,23810,NaN,NaN,443662,4438316,2464408.0,24.0,NaN,NaN
3,2020-12-13T17:00:00,4,Trentino-Alto Adige,634,74,708,12406,13114,56,408,30620,1405,0.0,0.0,45139,747597,300079.0,4.0,0,NaN
4,2020-12-13T17:00:00,5,Veneto,2606,340,2946,87753,90699,2567,4092,95141,4801,NaN,NaN,190641,2997581,1125556.0,13.0,NaN,NaN
5,2020-12-13T17:00:00,6,Friuli Venezia Giulia,655,60,715,13458,14173,-182,573,24683,1233,NaN,NaN,40089,815900,310500.0,3.0,NaN,NaN
6,2020-12-13T17:00:00,7,Liguria,768,75,843,7552,8395,-74,331,44566,2657,NaN,NaN,55618,649185,304928.0,4.0,i 113911 test antigenici rapidi sono riferiti ...,NaN
7,2020-12-13T17:00:00,8,Emilia-Romagna,2819,220,3039,61723,64762,790,1940,73421,6572,NaN,NaN,144755,2323522,1150185.0,14.0,NaN,NaN
8,2020-12-13T17:00:00,9,Toscana,1180,219,1399,16715,18114,-978,673,91079,3151,NaN,NaN,112344,1718159,1005743.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN
9,2020-12-13T17:00:00,10,Umbria,294,45,339,4469,4808,-76,145,20856,526,NaN,NaN,26190,456106,236951.0,1.0,NaN,NaN


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,...,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,0,2020-12-12T17:00:00,1,Piemonte,3857,304,4161,53700,57861,-2130,...,119614,7052,NaN,NaN,184527,1749139,1032439.0,6.0,NaN,NaN
1,1,2020-12-12T17:00:00,2,Valle d'Aosta,76,8,84,579,663,-53,...,5823,351,NaN,NaN,6837,57784,35186.0,0.0,NaN,NaN
2,2,2020-12-12T17:00:00,3,Lombardia,5289,717,6006,79060,85066,-2383,...,332595,23666,NaN,NaN,441327,4412793,2455811.0,26.0,NaN,NaN
3,19,2020-12-12T17:00:00,4,Trentino-Alto Adige,645,71,716,12342,13058,-32,...,30274,1399,0.0,0.0,44731,743590,299331.0,4.0,0,NaN
4,3,2020-12-12T17:00:00,5,Veneto,2609,345,2954,85178,88132,3039,...,93648,4769,NaN,NaN,186549,2981631,1120408.0,54.0,NaN,NaN
5,4,2020-12-12T17:00:00,6,Friuli Venezia Giulia,654,59,713,13642,14355,120,...,23946,1215,NaN,NaN,39516,809115,306882.0,8.0,NaN,NaN
6,5,2020-12-12T17:00:00,7,Liguria,782,77,859,7610,8469,-296,...,44169,2649,NaN,NaN,55287,646433,304092.0,4.0,i 112175 test antigenici rapidi sono riferiti ...,NaN
7,6,2020-12-12T17:00:00,8,Emilia-Romagna,2813,222,3035,60937,63972,-83,...,72315,6529,NaN,NaN,142816,2312385,1144793.0,14.0,NaN,NaN
8,7,2020-12-12T17:00:00,9,Toscana,1221,228,1449,17643,19092,-1732,...,89462,3117,NaN,NaN,111671,1705743,1001493.0,11.0,Positivi diagnosticati solo con test antigenic...,NaN
9,8,2020-12-12T17:00:00,10,Umbria,301,49,350,4534,4884,-205,...,20644,517,NaN,NaN,26045,453903,236236.0,0.0,NaN,NaN


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione
0,2020-12-13T17:00:00,1,Piemonte,3835,291,4126,51848,55974,-1887,1011,...,7088,NaN,NaN,185538,1757469,1036540.0,6.0,Il numero dei tamponi antigenici positivi è so...,NaN,4341375
1,2020-12-13T17:00:00,2,Valle d'Aosta,74,8,82,540,622,-41,34,...,353,NaN,NaN,6871,58056,35365.0,0.0,NaN,NaN,125501
2,2020-12-13T17:00:00,3,Lombardia,5159,714,5873,80476,86349,1283,2335,...,23810,NaN,NaN,443662,4438316,2464408.0,24.0,NaN,NaN,10103969
3,2020-12-13T17:00:00,4,Trentino-Alto Adige,634,74,708,12406,13114,56,408,...,1405,0.0,0.0,45139,747597,300079.0,4.0,0,NaN,1074819
4,2020-12-13T17:00:00,5,Veneto,2606,340,2946,87753,90699,2567,4092,...,4801,NaN,NaN,190641,2997581,1125556.0,13.0,NaN,NaN,4907704
5,2020-12-13T17:00:00,6,Friuli Venezia Giulia,655,60,715,13458,14173,-182,573,...,1233,NaN,NaN,40089,815900,310500.0,3.0,NaN,NaN,1211357
6,2020-12-13T17:00:00,7,Liguria,768,75,843,7552,8395,-74,331,...,2657,NaN,NaN,55618,649185,304928.0,4.0,i 113911 test antigenici rapidi sono riferiti ...,NaN,1543127
7,2020-12-13T17:00:00,8,Emilia-Romagna,2819,220,3039,61723,64762,790,1940,...,6572,NaN,NaN,144755,2323522,1150185.0,14.0,NaN,NaN,4467118
8,2020-12-13T17:00:00,9,Toscana,1180,219,1399,16715,18114,-978,673,...,3151,NaN,NaN,112344,1718159,1005743.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729
9,2020-12-13T17:00:00,10,Umbria,294,45,339,4469,4808,-76,145,...,526,NaN,NaN,26190,456106,236951.0,1.0,NaN,NaN,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi,popolazione,totale_posti_terapia_intensiva
0,2020-12-13T17:00:00,1,Piemonte,3835,291,4126,51848,55974,-1887,1011,...,NaN,NaN,185538,1757469,1036540.0,6.0,Il numero dei tamponi antigenici positivi è so...,NaN,4341375,575
1,2020-12-13T17:00:00,2,Valle d'Aosta,74,8,82,540,622,-41,34,...,NaN,NaN,6871,58056,35365.0,0.0,NaN,NaN,125501,20
2,2020-12-13T17:00:00,3,Lombardia,5159,714,5873,80476,86349,1283,2335,...,NaN,NaN,443662,4438316,2464408.0,24.0,NaN,NaN,10103969,1036
3,2020-12-13T17:00:00,4,Trentino-Alto Adige,634,74,708,12406,13114,56,408,...,0.0,0.0,45139,747597,300079.0,4.0,0,NaN,1074819,106
4,2020-12-13T17:00:00,5,Veneto,2606,340,2946,87753,90699,2567,4092,...,NaN,NaN,190641,2997581,1125556.0,13.0,NaN,NaN,4907704,1016
5,2020-12-13T17:00:00,6,Friuli Venezia Giulia,655,60,715,13458,14173,-182,573,...,NaN,NaN,40089,815900,310500.0,3.0,NaN,NaN,1211357,180
6,2020-12-13T17:00:00,7,Liguria,768,75,843,7552,8395,-74,331,...,NaN,NaN,55618,649185,304928.0,4.0,i 113911 test antigenici rapidi sono riferiti ...,NaN,1543127,209
7,2020-12-13T17:00:00,8,Emilia-Romagna,2819,220,3039,61723,64762,790,1940,...,NaN,NaN,144755,2323522,1150185.0,14.0,NaN,NaN,4467118,634
8,2020-12-13T17:00:00,9,Toscana,1180,219,1399,16715,18114,-978,673,...,NaN,NaN,112344,1718159,1005743.0,10.0,Positivi diagnosticati solo con test antigenic...,NaN,3722729,523
9,2020-12-13T17:00:00,10,Umbria,294,45,339,4469,4808,-76,145,...,NaN,NaN,26190,456106,236951.0,1.0,NaN,NaN,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-12-13T17:00:00,5,Veneto,2606,340,2946,87753,90699,2567,4092,...,33.46,2575,-1006,15950,5148.0,32,1493,25.66,0.08338,3.88
1,2020-12-13T17:00:00,3,Lombardia,5159,714,5873,80476,86349,1283,2335,...,68.92,1416,-401,25523,8597.0,144,908,9.15,0.02311,4.39
2,2020-12-13T17:00:00,8,Emilia-Romagna,2819,220,3039,61723,64762,790,1940,...,34.70,786,133,11137,5392.0,43,1106,17.42,0.04343,3.24
3,2020-12-13T17:00:00,12,Lazio,2937,341,3278,83331,86609,-709,1339,...,40.26,-681,145,14815,8735.0,27,2021,9.04,0.02283,2.38
4,2020-12-13T17:00:00,15,Campania,1804,134,1938,91322,93260,-829,1219,...,26.53,-836,-195,17319,10032.0,35,2013,7.04,0.02107,3.01
5,2020-12-13T17:00:00,16,Puglia,1547,191,1738,50408,52146,910,1175,...,51.76,953,-303,7122,1924.0,23,242,16.50,0.02931,1.82
6,2020-12-13T17:00:00,1,Piemonte,3835,291,4126,51848,55974,-1887,1011,...,50.61,-1852,-432,8330,4101.0,36,2862,12.14,0.02329,4.27
7,2020-12-13T17:00:00,19,Sicilia,1226,198,1424,34295,35719,-42,808,...,33.67,-27,-208,7094,4385.0,21,829,11.39,0.01626,1.57
8,2020-12-13T17:00:00,9,Toscana,1180,219,1399,16715,18114,-978,673,...,41.87,-928,99,12416,4250.0,34,1617,5.42,0.01808,3.02
9,2020-12-13T17:00:00,20,Sardegna,589,59,648,14776,15424,434,633,...,32.78,433,465,3606,3180.0,7,192,17.55,0.03882,1.61


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Trentino-Alto Adige,74,32,106,69.81
1,Lombardia,714,322,1036,68.92
2,Marche,81,62,143,56.64
3,Puglia,191,178,369,51.76
4,Piemonte,291,284,575,50.61
5,Abruzzo,63,85,148,42.57
6,Toscana,219,304,523,41.87
7,Lazio,341,506,847,40.26
8,Valle d'Aosta,8,12,20,40.00
9,Umbria,45,72,117,38.46


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv


In [18]:
df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
df_tb

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,21,P.A. Bolzano,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
2,2020-02-25T18:00:00,21,P.A. Bolzano,1,0,1,0,1,1,1,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
3,2020-02-25T18:00:00,22,P.A. Trento,0,0,0,0,0,0,0,0,0,NaN,NaN,0,3,NaN,NaN,NaN,NaN
4,2020-02-26T18:00:00,21,P.A. Bolzano,1,0,1,0,1,0,0,0,0,NaN,NaN,1,1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
583,2020-12-11T17:00:00,22,P.A. Trento,411,47,458,1981,2439,-7,160,14783,754,NaN,NaN,17976,406811,143863.0,1.0,NaN,"Dei 160 casi odierni, 40 sono molecolari con p..."
584,2020-12-12T17:00:00,21,P.A. Bolzano,242,27,269,10353,10622,-29,250,15277,634,NaN,NaN,26533,333319,154758.0,2.0,NaN,NaN
585,2020-12-12T17:00:00,22,P.A. Trento,403,44,447,1989,2436,-3,222,14997,765,NaN,NaN,18198,410271,144573.0,2.0,NaN,"Dei 222 casi odierni, 72 sono molecolari con p..."
586,2020-12-13T17:00:00,21,P.A. Bolzano,221,26,247,10352,10599,-23,251,15547,638,NaN,NaN,26784,335303,155286.0,0.0,NaN,NaN


In [19]:
# Initialize df_taa
df_taa = Functions.replace_bt_with_taa(df_tb[:2])
for i in range(2, df_tb.shape[0], 2):
    df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
df_taa = add_cols(df_taa)
df_taa

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,nuovi_deceduti,nuovi_tamponi,nuovi_casi_testati,variazione_isolamento_domiciliare,ti_su_to,to_su_tp,np_su_nt,np_su_nct,var_ti_perc,var_pos_perc
1,2020-02-25T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
2,2020-02-26T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
3,2020-02-27T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,30.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
4,2020-02-28T18:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,0.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
5,2020-02-29T17:00:00,4,Trentino-Alto Adige,1,0,1,0,1,0,0,...,0.0,25.0,0.0,0.0,0.00,100.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,2020-12-09T17:00:00,4,Trentino-Alto Adige,827,78,905,12495,13400,60,414,...,26.0,4136.0,750.0,142.0,8.62,6.75,10.01,55.20,0.00,0.45
290,2020-12-10T17:00:00,4,Trentino-Alto Adige,670,79,749,12418,13167,-233,461,...,17.0,4673.0,1452.0,-77.0,10.55,5.69,9.87,31.75,1.28,-1.74
291,2020-12-11T17:00:00,4,Trentino-Alto Adige,653,74,727,12363,13090,-77,412,...,27.0,4816.0,829.0,-55.0,10.18,5.55,8.55,49.70,-6.33,-0.58
292,2020-12-12T17:00:00,4,Trentino-Alto Adige,645,71,716,12342,13058,-32,472,...,16.0,5854.0,1384.0,-21.0,9.92,5.48,8.06,34.10,-4.05,-0.24


In [20]:
region_name = df_taa.denominazione_regione.iloc[0].lower()
csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
df_taa.to_csv(csv_out_region_name, index=False)
print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-trentino-alto adige.csv
